In [43]:
import numpy as np
import json
from pathlib import Path
import urllib
import cv2
from random import sample
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.mixture import BayesianGaussianMixture

In [44]:
ANNOTATIONS_PATH = Path("./data/laser_annotations.json")
CACHE_PATH = Path("./cache")

CACHE_PATH.mkdir(parents=True, exist_ok=True)

In [45]:
with open(ANNOTATIONS_PATH, "r") as f:
    annotations = json.load(f)

annotations

[{'id': 33564,
  'annotations': [{'id': 58,
    'completed_by': 4,
    'result': [{'original_width': 3987,
      'original_height': 3016,
      'image_rotation': 0,
      'value': {'x': 47.58695457488871,
       'y': 39.99424912201776,
       'width': 0.22547197916327916,
       'keypointlabels': ['Red Laser']},
      'id': 'E6WkooRAi_',
      'from_name': 'kp-1',
      'to_name': 'img-1',
      'type': 'keypointlabels',
      'origin': 'manual'}],
    'was_cancelled': False,
    'ground_truth': False,
    'created_at': '2024-10-24T15:12:58.018502Z',
    'updated_at': '2024-10-24T15:12:58.018514Z',
    'draft_created_at': '2024-10-24T15:12:57.018123Z',
    'lead_time': 15.71,
    'prediction': {},
    'result_count': 0,
    'unique_id': 'b0d60270-e8a6-4519-889d-b5782a0ecdeb',
    'import_id': None,
    'last_action': None,
    'task': 33564,
    'project': 10,
    'updated_by': 4,
    'parent_prediction': None,
    'parent_annotation': None,
    'last_created_by': None}],
  'drafts': [

In [46]:
jpg_files = [Path(urllib.parse.unquote(a["data"]["img"]).replace("/data/local-files/?d=fs_png_labeling_project_laser/REEF/data", "/home/ubuntu/mnt/fishsense_data/REEF/results/preprocess_disablehist")).with_suffix(".jpg") for a in annotations if a["annotations"] and a["annotations"][0]["result"]]

jpg_files

[PosixPath('/home/ubuntu/mnt/fishsense_data/REEF/results/preprocess_disablehist/2023-09-07 REEF Data Dump/080123_FSL-01 Photos/P8010001.jpg'),
 PosixPath('/home/ubuntu/mnt/fishsense_data/REEF/results/preprocess_disablehist/2023-09-07 REEF Data Dump/080123_FSL-01 Photos/P8010002.jpg'),
 PosixPath('/home/ubuntu/mnt/fishsense_data/REEF/results/preprocess_disablehist/2023-09-07 REEF Data Dump/080123_FSL-01 Photos/P8010003.jpg'),
 PosixPath('/home/ubuntu/mnt/fishsense_data/REEF/results/preprocess_disablehist/2023-09-07 REEF Data Dump/080123_FSL-01 Photos/P8010004.jpg'),
 PosixPath('/home/ubuntu/mnt/fishsense_data/REEF/results/preprocess_disablehist/2023-09-07 REEF Data Dump/080123_FSL-01 Photos/P8010005.jpg'),
 PosixPath('/home/ubuntu/mnt/fishsense_data/REEF/results/preprocess_disablehist/2023-09-07 REEF Data Dump/080123_FSL-01 Photos/P8010006.jpg'),
 PosixPath('/home/ubuntu/mnt/fishsense_data/REEF/results/preprocess_disablehist/2023-09-07 REEF Data Dump/080123_FSL-01 Photos/P8010007.jpg'),

In [47]:
width = annotations[0]["annotations"][0]["result"][0]["original_width"]
height = annotations[0]["annotations"][0]["result"][0]["original_height"]

width, height

(3987, 3016)

In [48]:
lasers = [(a["annotations"][0]["result"][0]["value"]["x"] / 100 * width, a["annotations"][0]["result"][0]["value"]["y"] / 100 * height) for a in annotations if a["annotations"] and a["annotations"][0]["result"]]

lasers

[(1897.2918789008127, 1206.2265535200556),
 (1902.584408016468, 1209.195867285987),
 (1902.525227162296, 1212.6337369056105),
 (1903.88638680825, 1210.4978460777695),
 (1912.5154209633727, 1304.4989994738762),
 (1912.4814487980575, 1302.6747403593677),
 (1896.0969476737005, 1247.2824879387545),
 (1895.5913578068999, 1243.4615818515274),
 (1925.661100901855, 1343.173984661684),
 (1926.583480381989, 1344.4538347022258),
 (1926.0233907748052, 1345.0648415464266),
 (1925.7433459712129, 1345.3499780737204),
 (1924.4958736643027, 1341.5006349552548),
 (1925.9538356600256, 1342.3889856999715),
 (1937.703667469905, 1328.414044540647),
 (1937.9657972750956, 1325.623714685489),
 (1926.9755431070175, 1289.7778872489082),
 (1925.6201262576317, 1285.0339282760583),
 (1923.5870009835533, 1283.0008030019796),
 (1923.2789516996024, 1278.19523417234),
 (1947.029054650777, 1393.3874387502158),
 (1946.609035323943, 1390.5942252026064),
 (1945.9469290387608, 1392.0579701411677),
 (1944.6823879664346, 1387

In [90]:
LASER_CACHE_PATH = CACHE_PATH / "laser_colors.npz"
OTHER_CACHE_PATH = CACHE_PATH / "other_colors.npz"

if not LASER_CACHE_PATH.exists() or not OTHER_CACHE_PATH.exists():
    laser_colors = []
    other_colors = []

    for path, (x, y) in zip(tqdm(jpg_files), lasers):
        img = cv2.imread(path.as_posix())
        laser_mask = np.zeros((height, width), dtype=bool)

        laser_mask[(int(y) - 10):(int(y) + 10), (int(x) - 10):(int(x) + 10)] = 1
        count = laser_mask.sum()    

        laser_colors.append(img[laser_mask, :])
        other_colors.append(sample(list(img[laser_mask == 0, :]), count))

    laser_colors = np.array(laser_colors)
    other_colors = np.array(other_colors)

    np.savez_compressed(LASER_CACHE_PATH, laser_colors)
    np.savez_compressed(OTHER_CACHE_PATH, other_colors)

100%|██████████| 2680/2680 [1:22:07<00:00,  1.84s/it]


In [97]:
laser_colors = np.load(LASER_CACHE_PATH)["arr_0"]
other_colors = np.load(OTHER_CACHE_PATH)["arr_0"]

In [98]:
annocation_count, _, _ = laser_colors.shape

laser_colors_flat = laser_colors.reshape((annocation_count, count * 3))
other_colors_flat = other_colors.reshape((annocation_count, count * 3))

In [99]:
laser_colors_flat.shape, other_colors_flat.shape

((2680, 1200), (2680, 1200))

In [100]:
laser_train, laser_test, other_train, other_test = train_test_split(laser_colors_flat, other_colors_flat, test_size=0.33)

laser_train.shape, laser_test.shape, other_train.shape, other_test.shape

((1795, 1200), (885, 1200), (1795, 1200), (885, 1200))

In [101]:
laser_gm = BayesianGaussianMixture(n_components=1200, random_state=0, max_iter=1000).fit(laser_train)
other_gm = BayesianGaussianMixture(n_components=1200, random_state=0, max_iter=1000).fit(other_train)

In [102]:
laser_test_laser_res = laser_gm.score_samples(laser_test)
laser_test_other_res = other_gm.score_samples(laser_test)

(laser_test_laser_res > laser_test_other_res).sum() / laser_test_laser_res.shape

array([0.99096045])